<a href="https://colab.research.google.com/github/PHRCpvh/BERTEmbedding/blob/master/Sentimentos_com_BERT_e_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 - Importação de Bibliotecas

In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random
from google.colab import drive

In [ ]:
!pip install bert-for-tf2

In [ ]:
!pip install sentencepiece

In [ ]:
%tensorflow_version 2.x

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import bert

In [ ]:
tf.__version__
!#pip install tensorflow==2.2.0-rc3

In [ ]:
tf.__version__

'2.2.0-rc3'

# 2 - Pré-processamento

Carregamento da base de dados

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wE6rGhNFuGPOc8KywtVBt3QWeS1WaR3a2jug5g7Z3QKV9NMeTWM6bA
Mounted at /gdrive
/gdrive


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
cols = ['sentiment', 'id', 'date', 'query', 'user', 'text']

In [ ]:
data = pd.read_csv("/content/drive/My Drive/Google Colab/training.1600000.processed.noemoticon.csv",
                   header = None,
                   names = cols,
                   engine = 'python',
                   encoding = 'latin1'
                   )

In [ ]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [ ]:
data.shape

(1600000, 2)

In [ ]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


Pré-processamento

Limpeza de textos

In [ ]:
def clean_tweet(tweet):
  tweet = BeautifulSoup(tweet, 'lxml').get_text()
  tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
  tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
  tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
  tweet = re.sub(r" +", ' ', tweet)
  return tweet

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

Gen RAM Free: 4.1 GB  | Proc size: 9.4 GB
GPU RAM Free: 10352MB | Used: 4727MB | Util  31% | Total 15079MB


In [ ]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [ ]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

Tokenização

In [ ]:
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [ ]:
def encode_sentence(sent):
  return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [ ]:
encode_sentence("My dog likes strawberries.")

['[CLS]', 'my', 'dog', 'likes', 'straw', '##berries', '.', '[SEP]']

In [ ]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

In [ ]:
print(data_inputs[0:2])

[['[CLS]', 'aw', '##w', '##w', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', 'd', '[SEP]'], ['[CLS]', 'is', 'upset', 'that', 'he', 'can', "'", 't', 'update', 'his', 'facebook', 'by', 'text', '##ing', 'it', '.', '.', '.', 'and', 'might', 'cry', 'as', 'a', 'result', 'school', 'today', 'also', '.', 'blah', '!', '[SEP]']]


Criação da Base de Dados

In [ ]:
def get_ids(tokens):
  return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
get_ids(tokenizer.tokenize("My dog likes strawberries."))

[2026, 3899, 7777, 13137, 20968, 1012]

In [ ]:
np.char.not_equal("[PAD]", "[PAD]")

array(False)

In [ ]:
def get_mask(tokens):
  return np.char.not_equal(tokens, "[PAD]").astype(int)

In [ ]:
get_mask(tokenizer.tokenize("My dog likes strawberries."))

array([1, 1, 1, 1, 1, 1])

In [ ]:
def get_segments(tokens):
  seg_ids = []
  current_seg_id = 0
  for tok in tokens:
    seg_ids.append(current_seg_id)
    if tok == "[SEP]":
      current_seg_id = 1 - current_seg_id
  return seg_ids

In [ ]:
print(data_inputs[0])

['[CLS]', 'aw', '##w', '##w', 'that', "'", 's', 'a', 'bum', '##mer', '.', 'you', 'should', '##a', 'got', 'david', 'carr', 'of', 'third', 'day', 'to', 'do', 'it', '.', 'd', '[SEP]']


In [ ]:
get_segments(data_inputs[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
my_sent = ["[CLS]"] + tokenizer.tokenize("Roses are Red.") + ["[SEP]"]
my_sent

['[CLS]', 'roses', 'are', 'red', '.', '[SEP]']

In [ ]:
bert_layer([
            tf.expand_dims(tf.cast(get_ids(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_mask(my_sent), tf.int32), 0),
            tf.expand_dims(tf.cast(get_segments(my_sent), tf.int32), 0),
           ])

[<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
 array([[-9.27935600e-01, -4.10335511e-01, -9.65755105e-01,
          9.07317996e-01,  8.12913895e-01, -1.74174443e-01,
          9.11234558e-01,  3.41952264e-01, -8.74521315e-01,
         -9.99989271e-01, -7.78410137e-01,  9.69385266e-01,
          9.86160517e-01,  6.36963308e-01,  9.48631346e-01,
         -7.51193166e-01, -4.58339661e-01, -7.08104610e-01,
          4.62098390e-01, -6.57927096e-01,  7.60414600e-01,
          9.99994814e-01, -3.96861106e-01,  3.44166219e-01,
          6.16488695e-01,  9.94400024e-01, -7.76633859e-01,
          9.38316584e-01,  9.59452271e-01,  7.32879400e-01,
         -6.93436861e-01,  2.93080539e-01, -9.93785501e-01,
         -1.64551839e-01, -9.67019618e-01, -9.95549560e-01,
          5.32935500e-01, -6.88060999e-01,  1.34715335e-02,
          2.98195966e-02, -9.18356597e-01,  4.20526296e-01,
          9.99988914e-01,  2.52676398e-01,  6.06235743e-01,
         -3.50750118e-01, -1.00000000e+00,  4.975

In [ ]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key = lambda x: x[2])
sorted_all = [([get_ids(sent_lab[0]),
                get_mask(sent_lab[0]),
                get_segments(sent_lab[0])],
               sent_lab[1])
               for sent_lab in data_with_len if sent_lab[2] > 7]

In [ ]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [ ]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None,), ()),
                                       padding_values=(0,0))

In [ ]:
NB_BATCHES = len(sorted_all) // BATCH_SIZE
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

# 3 - Construção dos Modelos

In [ ]:
class DCNNBERTEmbedding(tf.keras.Model):

    def __init__(self,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 name="dcnn"):
        super(DCNNBERTEmbedding, self).__init__(name=name)

        self.bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                                         trainable = False)

        self.bigram = layers.Conv1D(filters = nb_filters,
                                    kernel_size = 2,
                                    padding='valid',
                                    activation='relu')
        self.trigram = layers.Conv1D(filters = nb_filters,
                                     kernel_size = 3,
                                     padding='valid',
                                     activation='relu')
        self.fourgram = layers.Conv1D(filters = nb_filters,
                                      kernel_size = 4,
                                      padding='valid',
                                      activation='relu')
        self.pool = layers.GlobalMaxPool1D()
        self.dense_1 = layers.Dense(units=FFN_units, activation='relu')
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation='sigmoid')
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation='softmax')

    def embed_with_bert(self, all_tokens):
      _, embs = self.bert_layer([all_tokens[:, 0, :],
                                 all_tokens[:, 1, :],
                                 all_tokens[:, 2, :]])
      return embs

    def call(self, inputs, training):
        x = self.embed_with_bert(inputs)
        
        x_1 = self.bigram(x)
        x_1 = self.pool(x_1)
        x_2 = self.trigram(x)
        x_2 = self.pool(x_2)
        x_3 = self.fourgram(x)
        x_3 = self.pool(x_3)

        merged = tf.concat([x_1, x_2, x_3], axis =-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)

        return output

# 4 - Treinamento

In [ ]:
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2
DROPOUT_RATE = 0.2
BATCH_SIZE = 32
NB_EPOCHS = 5

In [ ]:
Dcnn = DCNNBERTEmbedding(nb_filters = NB_FILTERS,
                         FFN_units = FFN_UNITS,
                         nb_classes = NB_CLASSES,
                         dropout_rate = DROPOUT_RATE)

In [ ]:
if NB_CLASSES == 2:
    Dcnn.compile(loss = "binary_crossentropy",
                 optimizer = "adam",
                 metrics = ["accuracy"])
else:
    Dcnn.compile(loss = "sparse_categorical_crossentropy",
                 optimizer = "adam",
                 metrics = ["sparse_categorical_accuracy"])

In [ ]:
checkpoint_path = '/content/drive/My Drive/Google Colab/Sentimentos com BERT/'

ckpt = tf.train.Checkpoint(Dcnn = Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print('Latest checkpoint restored!')

Latest checkpoint restored!


In [ ]:
class MyCustomCallBack(tf.keras.callbacks.Callback):
  
    def on_epoch_end(self, epoch, logs = None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [ ]:
history = Dcnn.fit(train_dataset,
                   steps_per_epoch = 100,
                   epochs = NB_EPOCHS,
                   callbacks = [MyCustomCallBack()])

Epoch 1/5
100/100 [==============================] - 5s 54ms/step - loss: 0.3778 - accuracy: 0.8309
Epoch 2/5
100/100 [==============================] - 5s 49ms/step - loss: 0.3977 - accuracy: 0.8225
Epoch 3/5
100/100 [==============================] - 11s 114ms/step - loss: 0.4239 - accuracy: 0.8094
Epoch 4/5
100/100 [==============================] - 19s 193ms/step - loss: 0.4139 - accuracy: 0.8147
Epoch 5/5
100/100 [==============================] - 7s 68ms/step - loss: 0.4216 - accuracy: 0.8141


Epoch 1/5
 99/100 [============================>.] - ETA: 0s - loss: 0.3794 - accuracy: 0.8302Checkpoint saved at /content/drive/My Drive/Google Colab/Sentimentos com BERT/.
100/100 [==============================] - 5s 54ms/step - loss: 0.3778 - accuracy: 0.8309

Epoch 2/5
 99/100 [============================>.] - ETA: 0s - loss: 0.3969 - accuracy: 0.8232Checkpoint saved at /content/drive/My Drive/Google Colab/Sentimentos com BERT/.
100/100 [==============================] - 5s 49ms/step - loss: 0.3977 - accuracy: 0.8225

Epoch 3/5
 99/100 [============================>.] - ETA: 0s - loss: 0.4226 - accuracy: 0.8103Checkpoint saved at /content/drive/My Drive/Google Colab/Sentimentos com BERT/.
100/100 [==============================] - 11s 114ms/step - loss: 0.4239 - accuracy: 0.8094

Epoch 4/5
100/100 [==============================] - ETA: 0s - loss: 0.4139 - accuracy: 0.8147Checkpoint saved at /content/drive/My Drive/Google Colab/Sentimentos com BERT/.
100/100 [==============================] - 19s 193ms/step - loss: 0.4139 - accuracy: 0.8147

Epoch 5/5
100/100 [==============================] - ETA: 0s - loss: 0.4216 - accuracy: 0.8141Checkpoint saved at /content/drive/My Drive/Google Colab/Sentimentos com BERT/.
100/100 [==============================] - 7s 68ms/step - loss: 0.4216 - accuracy: 0.8141

# 5 - Avaliação

In [ ]:
results = Dcnn.evaluate(test_dataset)
print(results)

4513/4513 [==============================] - 104s 23ms/step - loss: 0.4156 - accuracy: 0.8120
[0.41559892892837524, 0.8120360374450684]


In [ ]:
def get_prediction(sentence):
  tokens = encode_sentence(sentence)

  input_ids = get_ids(tokens)
  input_mask = get_mask(tokens)
  segment_ids = get_segments(tokens)

  inputs = tf.stack(
      [
       tf.cast(input_ids, dtype=tf.int32),
       tf.cast(input_mask, dtype=tf.int32),
       tf.cast(segment_ids, dtype=tf.int32),
      ], axis = 0)
  inputs = tf.expand_dims(inputs, 0)

  output = Dcnn(inputs, training=False)

  sentiment = math.floor(output*2)

  if sentiment == 0:
    print("Output of the model: {}\nPredicted sentiment: negative".format(output))
  elif sentiment == 1:
    print("Output of the model: {}\nPredicted sentiment: positive".format(output))

In [ ]:
get_prediction("This actor is horrible.")

Output of the model: [[0.04115984]]
Predicted sentiment: negative


In [ ]:
get_prediction("I like corgis.")

Output of the model: [[0.56070167]]
Predicted sentiment: positive
